In [18]:
from bs4 import BeautifulSoup
import requests
import webbrowser
import datetime

In [19]:
#get MLE data
url = "https://www.indeed.com/jobs"
querystring = {"q":"machine+learning+engineer","l":"San+Francisco+Bay+Area,+CA","explvl":"entry_level"}
response = requests.request("GET", url, params=querystring)
now = datetime.datetime.now()
mle_doc="indeed/mle_"+now.strftime("%Y-%m-%d_%H.%M")+".html"
#https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
with open(mle_doc, mode="wb+") as localfile:
    localfile.write(response.content)

In [20]:
#see MLE data
soup = BeautifulSoup(open(mle_doc), "html.parser")
raw_links = soup.find_all("a",class_="turnstileLink")

#other endpoints I found are /pagead and /cmp
acceptable_endpoints = ["rc"]
#https://stackoverflow.com/questions/36300158/python-split-text-after-the-second-occurrence-of-character
clickable_links=["https://www.indeed.com"+link["href"] for link in raw_links if link["href"].split("/", 2)[1]in acceptable_endpoints]
len(clickable_links),clickable_links

(9,
 ['https://www.indeed.com/rc/clk?jk=d933cd5f42554665&fccid=e3d1e9ce06e668e2&vjs=3',
  'https://www.indeed.com/rc/clk?jk=cfaf902153f61df0&fccid=f89deb5a97c7738a&vjs=3',
  'https://www.indeed.com/rc/clk?jk=501e9e1b2b7825c2&fccid=1f67744ae4e39ac9&vjs=3',
  'https://www.indeed.com/rc/clk?jk=010387cb5bf0ba34&fccid=f89deb5a97c7738a&vjs=3',
  'https://www.indeed.com/rc/clk?jk=30ba2304821766d9&fccid=2bbdd3fa4303bfdc&vjs=3',
  'https://www.indeed.com/rc/clk?jk=67290fc3fe835c8b&fccid=cdd85c4d445cce0f&vjs=3',
  'https://www.indeed.com/rc/clk?jk=41fb91abb2020639&fccid=c037aeb6086309d9&vjs=3',
  'https://www.indeed.com/rc/clk?jk=edd094aee9abcb0a&fccid=dd616958bd9ddc12&vjs=3',
  'https://www.indeed.com/rc/clk?jk=460159a790d57fb3&fccid=3ef6655314901dcd&vjs=3'])